# Generics

## Generic class

In [6]:
public class Repository<T>
{
    private List<T> items = new List<T>();

    public void Add(T item)
    {
        items.Add(item);
        Console.WriteLine("Item added to the repository.");
    }

    public void Remove(T item)
    {
        items.Remove(item);
        Console.WriteLine("Item removed from the repository.");
    }

    public void PrintAll()
    {
        foreach (var item in items)
        {
            Console.WriteLine(item.ToString());
        }
    }
}

public record Person2(string Name, int Age);

var repository = new Repository<Person2>();
repository.Add(new Person2("Antony", 33));

Item added to the repository.


## Generic method

In [7]:
public static void Swap<T>(ref T a, ref T b)
{
    T temp = a;
    a = b;
    b = temp;
}

public record Person2(string Name, int Age);
var person1 = new Person2("Andres", 20);
var person2 = new Person2("Camilo", 20);
Swap<Person2>(ref person1, ref person2);

## Generic interface

In [8]:
public interface IRepository<TKey, TValue> where TValue : IComparable<TKey>
{
    TValue GetByKey(TKey key);
    void Add(TKey key, TValue value);
    void Remove(TKey key);
    void PrintAll();
}

### Some Built in generic interfaces

In [9]:
IEnumerable<int> numbers = new List<int> { 1, 2, 3, 4, 5 };
foreach (int number in numbers)
{
    Console.WriteLine(number);
}

1
2
3
4
5


In [10]:
ICollection<string> names = new List<string>();
names.Add("John");
names.Add("Jane");
Console.WriteLine(names.Count);

2


In [11]:
IList<double> grades = new List<double> { 90.5, 85.2, 92.7 };
grades.Insert(1, 88.9);
double secondGrade = grades[1];
Console.WriteLine(secondGrade);

88.9


## Constraints

In [12]:
// public interface IRepository<T> : IDisposable
// {
// 	void Add(T newEntity);
// }

// public class SqlRepository<T> : IRepository<T> where T : class, IEntity
// {
// 	DbContext _ctx;
//     DbSet<T> _set;
    
//     public SqlRepository(DbContext ctx)
//     {
//     	_ctx = ctx;
//         _set = _ctx.Set<T>();
//     }

// 	public void Add(T newEntity)
//     {
//     	if(newEntity.IsValid())
//         {
//         	_set.Add(newEntity);
//         }
//     }
// }

In [94]:
public interface ILogger
{
    void Log(string message);
}

public class Logger<T> where T : ILogger
{
    private T logger;

    public Logger(T logger)
    {
        this.logger = logger;
    }

    public void LogMessage(string message)
    {
        logger.Log(message);
    }
}

public struct ConsoleLogger : ILogger
{
    public void Log(string message)
    {
        Console.WriteLine(message);
    }
}

Logger<ConsoleLogger> logger = new Logger<ConsoleLogger>(new ConsoleLogger());
logger.LogMessage("Hello, world!");

var logger2 = new ConsoleLogger();
logger.LogMessage("Hello, world!");

Hello, world!
Hello, world!


### Exercise

Write a generic sorting algorithm that can sort in ascending order an array of any type. The algorithm should use the IComparable interface to compare elements and arrange them.

In [95]:
#r "nuget: Dumpify"

using Dumpify;

public static class GenericSorter
{
    public static void Sort<T>(T[] array) where T : IComparable<T>
    {
        for (int i = 0; i < array.Length - 1; i++)
        {
            for (int j = i + 1; j < array.Length; j++)
            {
                if (array[i].CompareTo(array[j]) > 0)
                {
                    // Swap elements
                    T temp = array[i];
                    array[i] = array[j];
                    array[j] = temp;
                }
            }
        }
    }
}

int[] numbers = { 5, 2, 8, 1, 9 };
GenericSorter.Sort(numbers);
numbers.Dump();

string[] names = { "John", "Alice", "Bob", "David" };
GenericSorter.Sort(names);
names.Dump();

Installed Packages Dumpify, 0.5.4

## Contravariance and Covariance 

Covariance example

In [3]:
public class Account{ }
public class CheckingAccount: Account{ }
public class SavingAccount: Account{ }

IEnumerable<CheckingAccount> derivedAccounts = new List<CheckingAccount>(); 
IEnumerable<Account> accounts = derivedAccounts;
// There is no need to use Cast<T>() like in   derivedAccounts.Cast<Account>()

public interface IRepository<in T>{}
public class Repository<T> : IRepository<T>{}
IRepository<CheckingAccount> derivedAccountsRepo = new Repository<CheckingAccount>();
//IRepository<Account> accountsRepo = derivedAccountsRepo;

IRepository<Account> accountsRepo1 = new Repository<Account>();
IRepository<CheckingAccount> derivedAccountsRepo1 = accountsRepo1;

Contravariance example

In [4]:
    
public record Account(string SSN);
public record CheckingAccount(string Name, string SSN): Account(SSN);
public record SavingAccount(string Name, string SSN) : Account(SSN);

class AccountComparer : IComparer<Account> {
   public int Compare(Account x, Account y) {
      return string.CompareOrdinal(x.SSN, y.SSN);
   }
}

IComparer<Account> comparer1 = new AccountComparer();
IComparer<CheckingAccount> comparer2 = comparer1;
if (comparer2.Compare(new CheckingAccount("Sam", "14543CX"), 
                    new CheckingAccount("Paul","14543CX")) == 0) {
    Console.WriteLine("Both checking accounts belongs to same Person!");
}   

Both checking accounts belongs to same Person!


## Generics with nullable constraint

In [1]:
public class EmployeeRepository<T> where T : class
{
    public Dictionary<int, T> _employees = new Dictionary<int, T>();

    public void AddEmployee(int id, T employee)
    {
        _employees[id] = employee;
    }

    public T GetEmployeeById(int id)
    {
        if (_employees.TryGetValue(id, out T employee))
        {
            return employee;
        }

        return null;
    }
}

public class Employee
{
    public int Id {get; set;}
    public string Name {get; set;}
    public string Department {get; set;}
}

var repository = new EmployeeRepository<Employee>();

// Add some employees to the repository
repository.AddEmployee(1, new Employee { Id = 1, Name = "John Doe", Department = "Engineering" });
repository.AddEmployee(2, new Employee { Id = 2, Name = "Jane Smith", Department = "Marketing" });
repository.AddEmployee(3, null);
var employess = repository._employees;


## Exercise
Considering the advantages of using Covariance and contravariance, there is a general Repository implementation that I would like to use with some helper methods like WriteAllToConsole and AddManagers; that way we can avoid code repetition; I need to add support to use covariance and contravariance and take advantage of their capabilities when trying to reuse created repositories to execute common logic.
For the second part, create the Repository class using Reflection, you can use Employee, Person or Manager as type parameters.

In [2]:
#r "nuget: Dumpify"

using Dumpify;

public interface IEntity
{
    int Id { get; init; }
}

public record Person(int Id, string Name, int Age) : IEntity;
public record Employee(int Id, string Name, int Age, string Company)
 : Person(Id, Name, Age);

 public record Manager(int Id, string Name, int Age, string Company, int Ternure)
 : Employee(Id, Name, Age, Company);

public interface IRepository<T> 
{
    void Add (T newEntity);
    T FindById (int id);
    IList<T> RetrieveAll();
}

public class Repository<T> : IRepository<T> where T : IEntity
{
    private IList<T> _items;

    public Repository() => _items = new List<T>();

    public void Add (T newEntity) => _items.Add(newEntity);

    public T FindById (int id)
    {
        for(var i = 0; i < _items.Count(); i++)
        {
            if(id == _items[i].Id)
            {
                return _items[i];
            }
        }

        return default(T);
    }

    public IList<T> RetrieveAll() => _items;
}

private static void WriteAllToConsole(IRepository<IEntity> repository)
{
    var items = repository.RetrieveAll();
    foreach(var item in items)
    {
        item.Dump();
    }
}

private static void WriteEmployesToConsole(IRepository<Employee> repository)
{
    var items = repository.RetrieveAll();
    foreach(var item in items)
    {
        Console.Write("Employee: ");
        item.Dump();
    }
}

private static void AddManagers(IRepository<Manager> repository)
{
    repository.Add(new Manager(6, "Frank", 30, "Toyosa", 2));
    repository.Add(new Manager(7, "Robert", 30, "Toyosa", 2));
}

private static void AddEmployees(IRepository<Employee> repository)
{
    repository.Add(new Employee(4, "Louis", 30, "Toyota"));
    repository.Add(new Employee(5, "Louis", 30, "Toyota"));
;}

var mike = new Person(1, "Mike", 40);
var raul = new Person(2, "Raul", 50);

var employeeRepo = new Repository<Employee>();
//personRepo.Add(mike); // With this specific repository, we can't add a less derived object. This is fine
AddEmployees(employeeRepo);
WriteEmployesToConsole(employeeRepo);

// AddManagers(employeeRepo) //This does not work;
// WriteAllToConsole(employeeRepo) // This does not work;

Installed Packages Dumpify, 0.5.4

Employee:        Employee       
┌─────────┬──────────┐
│ Name    │ Value    │
├─────────┼──────────┤
│ Company │ "Toyota" │
│ Id      │ 4        │
│ Name    │ "Louis"  │
│ Age     │ 30       │
└─────────┴──────────┘

Employee:        Employee       
┌─────────┬──────────┐
│ Name    │ Value    │
├─────────┼──────────┤
│ Company │ "Toyota" │
│ Id      │ 5        │
│ Name    │ "Louis"  │
│ Age     │ 30       │
└─────────┴──────────┘

